# Creating a basic chat experience with kernel arguments

In this example, we show how you can build a simple chat bot by sending and updating the kernel arguments with your requests.

We introduce the Kernel Arguments object which in this demo functions similarly as a key-value store that you can use when running the kernel.

The chat history is local (i.e. in your computer's RAM) and not persisted anywhere beyond the life of this Jupyter session.

In future examples, we will show how to persist the chat history on disk so that you can bring it into your applications.

In this chat scenario, as the user talks back and forth with the bot, the chat context gets populated with the history of the conversation. During each new run of the kernel, the kernel arguments and chat history can provide the AI with its variables' content.


In [ ]:
!python -m pip install semantic-kernel==1.0.5

In [1]:
import sys
import os

# Add the samples directory to the Python path
samples_dir = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples'
sys.path.append(samples_dir)

try:
    from services import Service
    from service_settings import ServiceSettings

    service_settings = ServiceSettings(
        global_llm_service="AzureOpenAI")  # Example setting

    # Select a service to use for this notebook (available services: OpenAI, AzureOpenAI, HuggingFace)
    selectedService = (
        Service.AzureOpenAI
        if service_settings.global_llm_service is None
        else Service(service_settings.global_llm_service.lower())
    )
    print(f"Using service type: {selectedService}")
except ModuleNotFoundError as e:
    print(f"Error: {e}")
    print("Ensure that 'service_settings.py' is in the correct directory and properly named.")
    # Additional debugging information
    print("Current Python path:")
    for path in sys.path:
        print(path)

Using service type: Service.AzureOpenAI


In [2]:
from dotenv import load_dotenv
from semantic_kernel import Kernel

kernel = Kernel()


env_path = r'C:\Users\sulma\OneDrive\Documents\GitHub\semantic-kernel\python\samples\getting_started\.env'
load_dotenv(env_path)

service_id = None

# Load the API keys and other settings from the environment variables
openai_api_key = os.getenv('OPENAI_API_KEY')
openai_org_id = os.getenv('OPENAI_ORG_ID')
azure_openai_api_key = os.getenv('AZURE_OPENAI_API_KEY')
azure_chat_deployment_name = os.getenv('AZURE_OPENAI_DEPLOYMENT_NAME')
azure_openai_endpoint = os.getenv('AZURE_OPENAI_ENDPOINT')


if selectedService == Service.OpenAI:

    from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

    service_id = "default"

    kernel.add_service(


        OpenAIChatCompletion(service_id=service_id,
                             ai_model_id="gpt-3.5-turbo-1106"),


    )


elif selectedService == Service.AzureOpenAI:

    from semantic_kernel.connectors.ai.open_ai import AzureChatCompletion

    service_id = "default"

    kernel.add_service(


        AzureChatCompletion(
            service_id=service_id,
            api_key=azure_openai_api_key,           # Use the api_key from the .env file
            # Use the deployment_name from the .env file
            deployment_name=azure_chat_deployment_name,
            endpoint=azure_openai_endpoint,         # Use the endpoint from the .env file
            env_file_path=env_path,




        )
    )

Let's define a prompt outlining a dialogue chat bot.


In [3]:
prompt = """
ChatBot can have a conversation with you about any topic.
It can give explicit instructions or say 'I don't know' if it does not have an answer.

{{$history}}
User: {{$user_input}}
ChatBot: """

Register your semantic function


In [4]:
from semantic_kernel.connectors.ai.open_ai.prompt_execution_settings.open_ai_prompt_execution_settings import (
    OpenAIChatPromptExecutionSettings,
)
from semantic_kernel.prompt_template import PromptTemplateConfig
from semantic_kernel.prompt_template.input_variable import InputVariable

if selectedService == Service.OpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-3.5-turbo-1106",
        max_tokens=2000,
        temperature=0.7,
    )
elif selectedService == Service.AzureOpenAI:
    execution_settings = OpenAIChatPromptExecutionSettings(
        service_id=service_id,
        ai_model_id="gpt-35-turbo",
        max_tokens=2000,
        temperature=0.7,
    )

prompt_template_config = PromptTemplateConfig(
    template=prompt,
    name="chat",
    template_format="semantic-kernel",
    input_variables=[
        InputVariable(name="user_input", description="The user input", is_required=True),
        InputVariable(name="history", description="The conversation history", is_required=True),
    ],
    execution_settings=execution_settings,
)

chat_function = kernel.add_function(
    function_name="chat",
    plugin_name="chatPlugin",
    prompt_template_config=prompt_template_config,
)

In [11]:
from semantic_kernel.contents import ChatHistory

chat_history = ChatHistory()
chat_history.add_system_message("You are a helpful chatbot who is good about giving book recommendations.")

Initialize the Kernel Arguments


In [12]:
from semantic_kernel.functions import KernelArguments

arguments = KernelArguments(user_input="Hi, I'm looking for book suggestions", history=chat_history)

Chat with the Bot


In [13]:
response = await kernel.invoke(chat_function, arguments)
print(response)

Something went wrong in function invocation. During function invocation: 'chatPlugin-chat'. Error description: 'Error occurred while invoking function chat: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", RateLimitError("Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}"))'


KernelInvokeException: Error occurred while invoking function: 'chatPlugin-chat'

Update the history with the output


In [8]:
chat_history.add_assistant_message(str(response))

Keep Chatting!


In [9]:
async def chat(input_text: str) -> None:
    # Save new message in the context variables
    print(f"User: {input_text}")

    # Process the user message and get an answer
    answer = await kernel.invoke(chat_function, KernelArguments(user_input=input_text, history=chat_history))

    # Show the response
    print(f"ChatBot: {answer}")

    chat_history.add_user_message(input_text)
    chat_history.add_assistant_message(str(answer))

In [10]:
await chat("I love history and philosophy, I'd like to learn something new about Greece, any suggestion?")

User: I love history and philosophy, I'd like to learn something new about Greece, any suggestion?


Something went wrong in function invocation. During function invocation: 'chatPlugin-chat'. Error description: 'Error occurred while invoking function chat: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", RateLimitError("Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}"))'


KernelInvokeException: Error occurred while invoking function: 'chatPlugin-chat'

In [14]:
await chat("that sounds interesting, what is it about?")

User: that sounds interesting, what is it about?


Something went wrong in function invocation. During function invocation: 'chatPlugin-chat'. Error description: 'Error occurred while invoking function chat: ("<class 'semantic_kernel.connectors.ai.open_ai.services.azure_chat_completion.AzureChatCompletion'> service failed to complete the prompt", RateLimitError("Error code: 429 - {'error': {'code': '429', 'message': 'Requests to the ChatCompletions_Create Operation under Azure OpenAI API version 2024-02-01 have exceeded token rate limit of your current OpenAI S0 pricing tier. Please retry after 86400 seconds. Please go here: https://aka.ms/oai/quotaincrease if you would like to further increase the default rate limit.'}}"))'


KernelInvokeException: Error occurred while invoking function: 'chatPlugin-chat'

In [ ]:
await chat("if I read that book, what exactly will I learn about Greek history?")

In [ ]:
await chat("could you list some more books I could read about this topic?")

After chatting for a while, we have built a growing history, which we are attaching to each prompt and which contains the full conversation. Let's take a look!


In [ ]:
print(chat_history)